# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Bears and Fish</span>

##### Due 11:00 P.M. Wednesday Mar 9

##### <span style="color:red">Group Members:</span>

Group work is allowed for this project.  Write the name and DuckID for each member of the group in this markdown cell:

Name: Yifan Yu 951514326


Groups are exepcted to do more work.  See the sections at the end of this notebook for suggestions of how to extend the simulation.

##  <span style="color:teal;">Overview</span> 

The last project this term is a cellular automaton style biological simulation.  

The sytem is described in Chapter 11 of the Miller and Ranum text.  The simulation needs three class defintions:
* World defines a 2D grid where cells are accessed according to their row and column coordinates
* Fish defines one type of animal that can inhabit the world; fish can move and reproduce, and they can die from overcrowding
* Bear defines the other type of animal; bears can also move and reproduce, they eat fish, and they can die from starvation

**<span style="color:red">Hint:</span>** &nbsp; Before you start, think about what the Bear and Fish classes have in common, and consider defining a new class to use as the parent of both Bear and Fish.

##  <span style="color:teal;">Libraries</span> 

Your program will need to use `numpy` (a numeric processing library) and `random` (the builtin random number generator module).  You can import additional modules if you wish; add the import statements to this code cell.

In [1]:
import numpy as np
import random

##  <span style="color:teal;">Event Log</span> 

A useful debugging technique is to save descriptions of events in a log file.  For this project we'll just print messages so they appear in the output area of the code cell that calls the top level simulator.

If you want to see messages printed as your program runs set `logging` to True in the code cell below, and include calls to `log` at the appropriate places in your methods.

**Note:** &nbsp; you can change the definition of the `log` function so it writes to a file or  prints additional information.

In [2]:
logging = True

def log(message):
    if logging:
        print(message)

##  <span style="color:teal;">Classes, Functions, and Global Variables</span> 

As you write your code you can define additional classes, helper functions, and global variables.  Describe all of new classes, functions and variables in the following markdown cell, and put all of the definitions in one code cell right below the documentation.

Extra Credit: Add new item in the World! The Phytoplankton is food by fish, now fish is become a hunter! the Phytoplankton is harmless, but it will take place for the empty space, and if too much Phytoplankton, maybe bears can't move and wait to die! Once the Phytoplankton, they will breed new one! but if can't breed, they will die(overpopulation),so only fish can eat them, if bears eat too much fish, they will be trapped by Phytoplankton

In [96]:
class Phytoplankton:
    
    def __init__(self, world, pos):
        self.xpos = pos[0]
        self.ypos = pos[1]
        self.world = world
        self.world[pos]= self
        self.breedTick=0
        
    def getX(self):
        return self.xpos
    def getY(self):
        return self.ypos
   
    def location(self):
        return self.xpos, self.ypos
    
    def __repr__(self):
        return "Phytoplankton"
        
    def move(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        newx = self.xpos+ randomOffset[0]
        newy = self.ypos+ randomOffset[1]
        while not(0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            newx = self.xpos+ randomOffset[0]
            newy = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(newx,newy)== True:
            self.world.moveThing(self.xpos,self.ypos,newx,newy)
            self.xpos = newx
            self.ypos = newy
        
    def live(self):
        adjplant = 0
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        for Offset in offsetList:
            newx = self.xpos+ Offset[0]
            newy = self.ypos+ Offset[1]
            if (not self.world.emptylocaion(newx,newy)) and isinstance(self.world.lookAtLocation(newx,newy),Phytoplankton):
                self.trytoBreed()
            self.move()
    def trytoBreed(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        nex = self.xpos+ randomOffset[0]
        ney = self.ypos+ randomOffset[1]
        while not(0 <= nex < self.world.getMaxX() and 0 <= ney < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nex = self.xpos+ randomOffset[0]
            ney = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(nex,ney):
            newPhytoplankton= self.__class__(self.world,(nex,ney))
        else:
            self.world.delThing(self)
            

##  <span style="color:teal;">World</span>

Describe your World class in the following markdown cell, and write the code for the class in the code cell below the markdown cell.

The minimum requiremens for this class are:
* the constructor should be passed the grid size (number of rows and columns)
* all cells should contain `None` when the world is initialized; later they can contain references to Fish or Bear objects
* overload the methods that allow you to access a cell or store a value in a cell
* include a method named `biota` that returns a list of Fish and Bear objects currently in the grid

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Note:** &nbsp; There are no auto-grader tests for the World class; tests for this class will be done as part of the tests for the Bear and Fish classes.

YOUR ANSWER HERE

In [97]:
class World:
    
    def __init__(self,col,row):
        self.col = col
        self.row = row
        self.grid = [[None for j in range(col)]for i in range(row)]
        self.grid = np.array(self.grid)
        self.thinglist = []
        #log(self.grid)
        
    def __setitem__(self,pos,item):
        self.grid[pos] = item
        self.thinglist.append(item)
            
        
    def __getitem__(self,pos):
        item = self.grid[pos[0]][pos[1]]
        log(pos)  
        
    def emptylocaion(self,col,row):
        if self.grid[col][row] == None:
            return True
        else:
            return False
        
    def moveThing(self,oldx,oldy,nex,ney):
        self.grid[nex][ney] = self.grid[oldx][oldy]
        self.grid[oldx][oldy] = None
        
    def delThing(self,item):
        self.grid[item.getX()][item.getY()] = None
        self.thinglist.remove(item)
    
    def biota(self):     
        return self.thinglist
    
    def lookAtLocation(self,x,y):
        return self.grid[x][y]
    def getMaxX(self):
        return self.col
    def getMaxY(self):
        return self.row
    
        

### <span style="color:teal;">Fish</span>

Use the markdown cell and code cell below to implement and describe your Fish class.

During the simulation a Fish object needs to behave as follows:

(1) Fish are susceptible to overcrowding:  if there are fish in 2 or more neighboring cells the fish dies (it's removed from the simulation)

(2) A fish can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new fish is placed in that cell

(3) A fish can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the fish moves there

The constructor will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1 and 2 shown above
* `move` implements rule 3
* `location` returns the current grid location (row and column) of the object

Define a class variable named `breed_interval` to specify how many time steps a fish must be alive before it reproduces; the initial value for this variable is 12.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**NOTE:** &nbsp; The way the simulation is defined a fish might be eaten before the top level simulation calls the `live` method.  Your method should check to make sure the fish is still alive.  The easiest way to do this is to include an instance variable named `_alive` that is set to True when the fish is initialized and set to False when it dies.

YOUR ANSWER HERE

In [271]:
class Fish:
    breed_interval = 12
    survive_without_food = 6
    def __init__(self, world, pos):
        self.xpos = pos[0]
        self.ypos = pos[1]
        self.world = world
        self.world[pos]= self
        self.breedTick=0
        self.hungryTick = 0
        
    def getX(self):
        return self.xpos
    def getY(self):
        return self.ypos
   
    def location(self):
        return self.xpos, self.ypos
    
    def __repr__(self):
        return "fish"
        
    def move(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        newx = self.xpos+ randomOffset[0]
        newy = self.ypos+ randomOffset[1]
        while not(0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            newx = self.xpos+ randomOffset[0]
            newy = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(newx,newy)== True:
            self.world.moveThing(self.xpos,self.ypos,newx,newy)
            self.xpos = newx
            self.ypos = newy
        
    def live(self):
        self.breedTick +=1
        if self.breedTick>=self.breed_interval:
            self.trytoBreed()
        self.trytoEat()
        if self.hungryTick ==self.survive_without_food:
            self.world.delThing(self)
        else:
            self.move()
            
    def trytoBreed(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        nex = self.xpos+ randomOffset[0]
        ney = self.ypos+ randomOffset[1]
        while not(0 <= nex < self.world.getMaxX() and 0 <= ney < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nex = self.xpos+ randomOffset[0]
            ney = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(nex,ney):
            littlebear = self.__class__(self.world,(nex,ney))
            self.breedTick = 0
    
    def trytoEat(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        adjprey = []
        for offset in offsetList:
            newx = self.xpos + offset[0]
            newy = self.ypos + offset[1]
            if 0<= newx <self.world.getMaxX() and 0<= newy <self.world.getMaxY():
                if (not self.world.emptylocaion(newx,newy)) and isinstance(self.world.lookAtLocation(newx,newy),Phytoplankton):
                    adjprey.append(self.world.lookAtLocation(newx,newy))
        if len(adjprey)>0:
            randomprey = adjprey[random.randrange(len(adjprey))]
            preyx = randomprey.getX()
            preyy = randomprey.getY()
            self.world.delThing(randomprey)
            self.move()
            self.hungryTick = 0
        else:
            self.hungryTick +=1
        
        
        
        

In [272]:
w2 = World(5,5)

f2 = Fish(w2, (2,2))
f3 = Fish(w2, (2,1))
f1 = Fish(w2, (2,3))
print(w2.grid)

[[None None None None None]
 [None None None None None]
 [None fish fish fish None]
 [None None None None None]
 [None None None None None]]


In [249]:
w3 = World(5,5)
f3 = Fish(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
f3.live()
print(w3.biota())

[fish, fish, fish]


##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [250]:
# A new world has no objects
w1 = World(5,5)
assert len(w1.biota()) == 0

# After adding a fish there should be one object
f1 = Fish(w1, (2,2))
assert len(w1.biota()) == 1

# Test the location method
assert f1.location() == (2,2)

In [251]:
# Setting breed_interval to 1 should cause a fish to reproduce when live is called
w2 = World(5,5)
Fish.breed_interval = 1
f2 = Fish(w2, (2,2))
f2.live()
assert len(w2.biota()) == 2

# Reset the interval to original value for remaining tests
Fish.breed_interval = 12

In [252]:
# Make three fish, the one in the middle should die from overcrowding
w3 = World(5,5)
f3 = Fish(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
f3.live()
assert len(w2.biota()) == 2

In [253]:
# When a fish moves it should be within one cell of its original location
w4 = World(5,5)
f4 = Fish(w4, (2,2))
f4.move()
r, c = f4.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

###  <span style="color:teal;">Bear</span>

Use the markdown cell and code cell below to implement and describe your Bear class.

During the simulation a Bear object needs to behave as follows:

(1) The bear looks for fish in each adjacent cell; if it finds one or more fish it eats one at random 

(2) If a bear has not eaten for certain number of time steps it dies (it's removed from the simulation)

(3) A bear can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new bear is placed in that cell

(4) A bear can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the bear moves there

Define a class variable named `breed_interval` to specify how many time steps a bear must be alive before it reproduces; the initial value for this variable is 8.  Define another class variable named `survive_without_food` to be the number of time steps a bear can live before it dies from starvation; the initial value for this variable is 10.

The constructor will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1, 2, and 3 shown above
* `move` implements rule 4
* `location` returns the current grid location (row and column) of the object

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

YOUR ANSWER HERE

In [254]:
class Bear:
    breed_interval = 8
    survive_without_food = 10  
    def __init__(self, world, pos):
        self.xpos = pos[0]
        self.ypos = pos[1]
        self.world = world
        self.world[pos]= self
        self.breedTick = 0
        self.hungryTick = 0
        
    def getX(self):
        return self.xpos
    def getY(self):
        return self.ypos
   
    def location(self):
        return self.xpos, self.ypos
    
    def __repr__(self):
        return "bear"
        
    def move(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        newx = self.xpos+ randomOffset[0]
        newy = self.ypos+ randomOffset[1]
        while not(0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            newx = self.xpos+ randomOffset[0]
            newy = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(newx,newy)== True:
            self.world.moveThing(self.xpos,self.ypos,newx,newy)
            self.xpos = newx
            self.ypos = newy
        
    def live(self):
        self.breedTick +=1
        if self.breedTick>=self.breed_interval:
            self.trytoBreed()
        self.trytoEat()
        if self.hungryTick ==self.survive_without_food:
            self.world.delThing(self)
        else:
            self.move()
            
    def trytoBreed(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        nex = self.xpos+ randomOffset[0]
        ney = self.ypos+ randomOffset[1]
        while not(0 <= nex < self.world.getMaxX() and 0 <= ney < self.world.getMaxY()):
            randomOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nex = self.xpos+ randomOffset[0]
            ney = self.ypos+ randomOffset[1]
        
        if self.world.emptylocaion(nex,ney):
            littlebear = self.__class__(self.world,(nex,ney))
            self.breedTick = 0
    
    def trytoEat(self):
        offsetList = [(-1,1), (0,1), (1,1),(-1,0), (1,0), (-1,-1), (0,-1), (1,-1)]
        adjprey = []
        for offset in offsetList:
            newx = self.xpos + offset[0]
            newy = self.ypos + offset[1]
            if 0<= newx <self.world.getMaxX() and 0<= newy <self.world.getMaxY():
                if (not self.world.emptylocaion(newx,newy)) and isinstance(self.world.lookAtLocation(newx,newy),Fish):
                    adjprey.append(self.world.lookAtLocation(newx,newy))
        if len(adjprey)>0:
            randomprey = adjprey[random.randrange(len(adjprey))]
            preyx = randomprey.getX()
            preyy = randomprey.getY()
            self.world.delThing(randomprey)
            self.move()
            self.hungryTick = 0
        else:
            self.hungryTick +=1
       
    
    

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [255]:
# Test the Bear constructor and location method
w1 = World(5,5)
b1 = Bear(w1, (1,1))
assert len(w1.biota()) == 1
assert b1.location() == (1,1)

In [256]:
# Repeat the reproduction test for Bears
w2 = World(5,5)
Bear.breed_interval = 1
b2 = Bear(w2, (2,2))
b2.live()
assert len(w2.biota()) == 2
Bear.breed_interval = 8

In [257]:
# Make fish for the bear to eat, count the number of objects after eating
w3 = World(5,5)
b3 = Bear(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
b3.live()
assert len(w2.biota()) == 2

In [258]:
# Setting the survival limit to 1 should cause a Bear to starve 
w4 = World(5,5)
Bear.survive_without_food = 1
b4 = Bear(w4, (2,2))
b4.live()
assert len(w4.biota()) == 0

In [259]:
# Repeat the move test for bears
w5 = World(5,5)
b5 = Bear(w5, (2,2))
b5.move()
r, c = b5.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

##  <span style="color:teal;">Top Level Functions</span>

Fill in the body of the `wbf` function so it returns a new World object with the specified number of rows and columns and with the specified number of Bear and Fish objects at random locations.

When we grade your project we will call `wbf` to make a World object and then use the main loop (implemented by `step_system`) to run the simulation.

In [260]:
def wbf(nrows, ncols, nbears, nfish):
    myworld = World(ncols,nrows)    
    for i in range(nfish):
        x = random.randrange(myworld.getMaxX())
        y = random.randrange(myworld.getMaxY())
        
        while not myworld.grid[x][y]==None:
            x = random.randrange(myworld.getMaxX())
            y = random.randrange(myworld.getMaxY())    
        F = Fish(myworld,(x,y))
            
    for i in range(nbears):
        x = random.randrange(myworld.getMaxX())
        y = random.randrange(myworld.getMaxY())
        
        while not myworld.grid[x][y]==None:
            x = random.randrange(myworld.getMaxX())
            y = random.randrange(myworld.getMaxY())    
        B =Bear(myworld,(x,y))
    return myworld

In [261]:
w = wbf(10,10,3,12)
w.grid

array([[None, None, fish, None, None, None, None, None, None, None],
       [None, None, fish, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, fish, fish, None, bear, None],
       [None, None, None, None, None, None, None, None, None, bear],
       [None, None, None, None, None, None, fish, fish, None, None],
       [None, fish, None, None, None, bear, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, fish],
       [None, fish, fish, None, None, None, None, None, None, None],
       [fish, None, None, None, fish, None, None, None, None, None]], dtype=object)

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [262]:
w = wbf(10,10,3,12)

dct = { Bear: 0, Fish: 0 }
for x in w.biota():
    dct[x.__class__] += 1

assert dct[Bear] == 3
assert dct[Fish] == 12

##### <span style="color:red">Experiments</span>

Run some experiments with the top level simulation loop and describe the results in the markdown cell below.  Some ideas of things to try:
* The settings for the Bear class `breed_interval` and `survive_without_food` variables come from the textbook.  Will the world ever run out of bears with these settings?
* Change the settings so `Bear.breed_interval` is larger than `Bear.survive_without_food`.  How does that change the outcome?
* Set the `breed_interval` counter for the Fish class to a smaller number (e.g. 4) so the world has more fish.  What effect does that have?
* Write a function that runs the simulation for a specified number of generations, or until there are no more objects left in the grid.  What combination of parameters leads to the largest number of time steps before the simulation halts.

In [263]:
def step_system(world):
    for x in world.biota():
        x.live()
    for x in world.biota():
        x.move()

In [264]:
w = wbf(10,10,3,12)

In [270]:
step_system(w)
print(w.biota())

[fish, fish, fish, fish]


1)No, because the breed_interval is smaller than survive_without_food, after the food(fish)eat out, bears are hungry, but before they die,they breed new bear!

2)If breed_interval is bigger than survive_without_food, it will be faster to run out,if no more fish in the world.Now, they can seldom breed before they die, that means run out.

3)bears will live longer than before,but once bears all die, there will be all fishs easier.

4)fish breed_interval smaller, so that bears can have enough fish to eat, and let bears be easier to hungry or harder to breed, so that will control the population, and make a balance.

##  <span style="color:teal;">Projects for Groups</span>

Here are two suggestions for ways to extend the simulation.  We will consider other types of extensions -- send a request to `conery@uoregon.edu` with your proposal.

Groups with three people should implement two extensions.  Groups with two people can choose either extension, or do both for extra credit.

* Implement the Plant class described in Section 10.7 of the textbook, and modify the Fish class so fish eat plants and die if they don't find enough food.

* Implement a GUI using `tkinter` that is similar to the Solar System GUI.  A canvas should display the world along with images for the Bear and Fish objects (you can download `Bear.gif` and `Fish.gif` from Canvas).  Use a spinbox or text entry box to specify the number of Bear and Fish objects and the number of time steps to run.  Include a Run button to start the simulation.
